# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Zhang  ->  M. Zhang  |  ['M. Zhang']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
F. Walter  ->  F. Walter  |  ['F. Walter']


E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Arxiv has 56 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.21166


extracting tarball to tmp_2508.21166... done.
Retrieving document from  https://arxiv.org/e-print/2508.21533
extracting tarball to tmp_2508.21533...

 done.


Found 193 bibliographic references in tmp_2508.21533/aa55318-25corr.bbl.
Issues with the citations
syntax error in line 828: '}' expected
Retrieving document from  https://arxiv.org/e-print/2508.21576


extracting tarball to tmp_2508.21576... done.
Retrieving document from  https://arxiv.org/e-print/2508.21708
extracting tarball to tmp_2508.21708... done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/AGN.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{figures/AGN.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 157 bibliographic references in tmp_2508.21708/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2508.21818
extracting tarball to tmp_2508.21818... done.


Found 54 bibliographic references in tmp_2508.21818/main.bbl.
Error retrieving bib data for davies2025: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.21533-b31b1b.svg)](https://arxiv.org/abs/2508.21533) | **TOI-1438: A rare system with two short-period sub-Neptunes and a tentative long-period Jupiter-like planet orbiting a K0V star**  |
|| C. M. Persson, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2025-09-01*|
|*Comments*| *31 pages, 21 figures. Accepted by Astronomy & Astrophysics (A&A) 31 July 2025*|
|**Abstract**|            We present the detection and characterisation of the TOI-1438 multi-planet system discovered by TESS. We collected a series of follow-up observations including high-spectral resolution observations with HARPS-N over a period of five years. Our modelling shows that the K0V star hosts two transiting sub-Neptunes with Rb = 3.04 +/- 0.19 RE, Rc = 2.75 +/- 0.14 RE, Mb = 9.4 +/- 1.8 ME, and Mc = 10.6 +/- 2.1 ME. The orbital periods of planets b and c are 5.1 and 9.4 days, respectively, corresponding to instellations of 145 +/- 10 and 65 +/- 4 FE. The bulk densities are 1.8 +/- 0.5 and 2.9 +/- 0.7 g cm-3, respectively, suggesting a volatile-rich interior composition. We computed a set of planet interior structure models. Planet b presents a high-metallicity envelope that can accommodate up to 2.5 % in H/He in mass, while planet c cannot have more than 0.2 % as H/He in mass. For any composition of the core considered (Fe-rock or ice-rock), both planets would require a volatile-rich envelope. In addition to the two planets, the radial velocity (RV) data clearly reveal a third signal, likely coming from a non-transiting planet, with an orbital period of 7.6 +1.6 -2.4 years and a radial velocity semi-amplitude of 35+3-5 m s-1. Our best fit model finds a minimum mass of 2.1 +/- 0.3 MJ and an eccentricity of 0.25+0.08-0.11. However, several RV activity indicators also show strong signals at similar periods, suggesting this signal might (partly) originate from stellar activity. More data over a longer period of time are needed to conclusively determine the nature of this signal. If it is confirmed as a triple-planet system, TOI-1438 would be one of the few detected systems to date characterised by an architecture with two small, short-period planets and one massive, long-period planet, where the inner and outer systems are separated by an orbital period ratio of the order of a few hundred.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.21708-b31b1b.svg)](https://arxiv.org/abs/2508.21708) | **JWST Spectroscopic Insights Into the Diversity of Galaxies in the First 500 Myr: Short-Lived Snapshots Along a Common Evolutionary Pathway**  |
|| G. Roberts-Borsani, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-09-01*|
|*Comments*| *20 pages, 13 figures. Submitted for publication, comments welcome*|
|**Abstract**|            We investigate the nature and spectroscopic diversity of early galaxies from a sample of 40 sources at z>10 with JWST/NIRSpec prism observations, the largest of its kind thus far. We compare the properties of strong UV line emitters, as traced by intense CIV emission, with those of more "typical" sources with weak or undetected CIV. The more typical (or "CIV-weak") sources reveal significant scatter in their CIII] line strengths, UV continuum slopes, and physical sizes, spanning CIII] equivalent widths of ~1-51 Å, UV slopes of $\beta$~-1.6 to -2.6, and half-light radii of ~50-1000 pc. In contrast, CIV-strong sources generally occupy the tail of these distributions, with CIII] EWs of 16-51 Å, UV slopes $\beta$<-2.5, compact morphologies ($r_{50}$<100 pc), and elevated star formation surface densities ($\Sigma_{SFR}$>100 $M_{\odot}yr^{-1}kpc^{-2}$). Collectively these properties are consistent with concentrated starbursts that temporarily outshine the extended structure of the galaxy. Comparing average properties from composite spectra, we find the diversity of the sample is primarily driven by bursts and lulls of star formation on very short timescales (<3 Myr), where strong CIV emitters are observed at the apex of these phases and sources devoid of emission lines represent periods of relative inactivity. An apparent association between strong CIV and enhanced nitrogen abundance suggests both features may be modulated by the same duty cycle and reflect a generic mode of star formation. We show that AGN are unlikely to be a significant contributor to this duty cycle based on a comparison of UV line diagnostics to photoionisation models, although some non-thermal activity cannot be fully ruled out. Our results support a unified evolutionary picture whereby transient bursts and lulls can explain the spectral diversity and early growth of bright galaxies in the first 500 Myr.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.21818-b31b1b.svg)](https://arxiv.org/abs/2508.21818) | **First constraints on the local ionization topology in front of two quasars at z ~ 7.5**  |
|| T. Kist, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2025-09-01*|
|*Comments*| *submitted to MNRAS*|
|**Abstract**|            Thus far, Lyman-$\alpha$ damping wings towards quasars have been used to probe the \textit{global} ionization state of the foreground intergalactic medium (IGM). A new parameterization has demonstrated that the damping wing signature also carries \textit{local} information about the distribution of neutral hydrogen (HI) in front of the quasar before it started shining. Leveraging a recently introduced Bayesian \texttt{JAX}-based Hamiltonian Monte Carlo (HMC) inference framework, we derive constraints on the Lorentzian-weighted HI column density $N_\mathrm{HI}^\mathrm{DW}$, the quasar's distance $r_\mathrm{patch}$ to the first neutral patch and its lifetime $t_\mathrm{Q}$ based on JWST/NIRSpec spectra of the two $z \sim 7.5$ quasars J1007+2115 and J1342+0928. After folding in model-dependent topology information, we find that J1007+2115 (and J1342+0928) is most likely to reside in a $\langle x_\mathrm{HI} \rangle = 0.32_{-0.20}^{+0.22}$ ($0.58_{-0.23}^{+0.23}$) neutral IGM while shining for a remarkably short lifetime of $\log_{10} t_\mathrm{Q} /\mathrm{yr} = 4.14_{-0.18}^{+0.74}$ (an intermediate lifetime of $5.64_{-0.43}^{+0.25}$) along a sightline with $\log_{10} N_\mathrm{HI}^\mathrm{DW} /\mathrm{cm}^{-2} = 19.70_{-0.86}^{+0.35}$ ($20.24_{-0.22}^{+0.25}$) and $r_\mathrm{patch} = 28.9_{-14.4}^{+54.0} \,\mathrm{cMpc}$ ($10.9_{-5.9}^{+5.6} \,\mathrm{cMpc}$). In light of the potential presence of local absorbers in the foreground of J1342+0928 as has been recently suggested, we also demonstrate how the Lorentzian-weighted column density $N_\mathrm{HI}^\mathrm{DW}$ provides a natural means for quantifying their contribution to the observed damping wing signal.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.21166-b31b1b.svg)](https://arxiv.org/abs/2508.21166) | **Stringent Upper Bounds on Atmospheric Mass Loss from Three Neptune-Sized Planets in the TOI-4010 System**  |
|| M. Saidel, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-09-01*|
|*Comments*| *16 pages, 8 figures; Accepted for publication in the Astronomical Journal*|
|**Abstract**|            Photoevaporative models predict that the lower edge of the Neptune desert is sculpted by atmospheric mass loss. However, the stellar high energy fluxes that power hydrodynamic escape and set predicted mass loss rates can be uncertain by multiple orders of magnitude. These uncertainties can be bypassed by studying mass loss for planets within the same system, as they have effectively undergone scaled versions of the same irradiation history. The TOI-4010 system is an ideal test case for mass loss models, as it contains three Neptune-sized planets with planet b located in the `Neptune desert', planet c in the `Neptune ridge', and planet d in the `Neptune savanna'. Using Keck/NIRSPEC, we measured the metastable helium transit depths of all three planets in order to search for evidence of atmospheric escape. We place upper bounds on the excess helium absorption of 1.23\%, 0.81\%, and 0.87\% at 95\% confidence for TOI-4010~b, c and d respectively. We fit our transmission spectra with Parker wind models and find that this corresponds to 95th-percentile upper limits of $10^{10.17}$g~s$^{-1}$, $10^{10.53}$g~s$^{-1}$, and $10^{10.50}$g~s$^{-1}$ on the mass loss rates of TOI-4010~b, c, and d respectively. Our non-detections are inconsistent with expectations from one-dimensional hydrodynamic models for solar composition atmospheres. We consider potential reductions in signal from a decreased host star XUV luminosity, planetary magnetic fields, enhanced atmospheric metallicities, and fractionation, and explore the implications of our measurements for the past evaporation histories of all three planets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.21576-b31b1b.svg)](https://arxiv.org/abs/2508.21576) | **Tracing the Origins of Hot Halo Gas in Milky Way-Type Galaxies with SMUGGLE**  |
|| Z. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-01*|
|*Comments*| *13 pages, 7 figures, 1 table; ApJ accepted. Comments are welcome!*|
|**Abstract**|            Current galaxy formation models predict the existence of X-ray-emitting gaseous halos around Milky Way (MW)-type galaxies. To investigate properties of this coronal gas in MW-like galaxies, we analyze a suite of high-resolution simulations based on the {\it SMUGGLE} framework, and compare the results with X-ray observations of both the MW and external galaxies. We find that for subgrid models incorporating any form of stellar feedback, e.g., early feedback (including stellar winds and radiation) and/or supernova (SN) explosions, the total $0.5-2$\,keV luminosity is consistent {\it within uncertainties} with X-ray observations of the MW and with scaling relations derived for external disk galaxies. However, all models exhibit an X-ray surface brightness profile that declines too steeply beyond $\sim5$\,kpc, underpredicting the extended emission seen in recent eROSITA stacking results. Across all subgrid prescriptions, the simulated surface brightness and emission measure fall below MW observations by at least $1-2$ orders of magnitude, with the most severe discrepancy occurring in the no-feedback model. Our results suggest that (i) stellar feedback primarily shapes the innermost hot atmosphere (central $\sim5$\,kpc), with comparable contributions from early feedback and SNe to the resulting X-ray luminosity; (ii) additional mechanisms such as gravitational heating, AGN feedback, and/or Compton effects of GeV cosmic ray are necessary to generate the extended, volume-filling hot gaseous halo of MW-mass galaxies; (iii) the origins of hot corona in MW-like galaxies are partially distinct from those of the warm ($\sim10^5$\,K) gas, by combining our previous finding that the {\it SMUGGLE} model successfully reproduces the kinematics and spatial distribution of MW \ion{O}{6} absorbers \citep{2024ApJ...962...15Z}.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.21533/./Fig08a.png', 'tmp_2508.21533/./Fig08b.png', 'tmp_2508.21533/./Fig08c.png', 'tmp_2508.21533/./Fig08d.png', 'tmp_2508.21533/./Fig14.png', 'tmp_2508.21533/./Fig07.png']
copying  tmp_2508.21533/./Fig08a.png to _build/html/
copying  tmp_2508.21533/./Fig08b.png to _build/html/
copying  tmp_2508.21533/./Fig08c.png to _build/html/
copying  tmp_2508.21533/./Fig08d.png to _build/html/
copying  tmp_2508.21533/./Fig14.png to _build/html/
copying  tmp_2508.21533/./Fig07.png to _build/html/
exported in  _build/html/2508.21533.md
    + _build/html/tmp_2508.21533/./Fig08a.png
    + _build/html/tmp_2508.21533/./Fig08b.png
    + _build/html/tmp_2508.21533/./Fig08c.png
    + _build/html/tmp_2508.21533/./Fig08d.png
    + _build/html/tmp_2508.21533/./Fig14.png
    + _build/html/tmp_2508.21533/./Fig07.png
found figures ['tmp_2508.21708/./figures/stack_spectrum.png', 'tmp_2508.21708/./figures/uvslope_origins.png', 'tmp_2508.21708/./figures/bursty.png']
copying  tmp_2508.21708

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vsini}{V \sin i_\star}$
$\newcommand{\vrad}{V_{\rm rad}}$
$\newcommand{\vmic}{V_{\rm mic}}$
$\newcommand{\vmac}{V_{\rm mac}}$
$\newcommand{\dex}{\rm dex}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{log g_\star}$
$\newcommand{\met}{[M/H]}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\cah}{[Ca/H]}$
$\newcommand{\mgh}{[Mg/H]}$
$\newcommand{\nah}{[Na/H]}$
$\newcommand{\sih}{[Si/H]}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\ms}{m~s^{-1}}$
$\newcommand{\msd}{m~s^{-1}~day^{-1}}$
$\newcommand{\gc}{g~cm^{-3}}$
$\newcommand{\lgr}{\log (R^\prime_{HK})}$
$\newcommand{\lgt}{\log~t}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\msunyr}{M_{\odot} yr^{-1}}$
$\newcommand{\Fearth}{F_{\oplus}}$
$\newcommand{\mearth}{M_{\oplus}}$
$\newcommand{\rearth}{R_{\oplus}}$
$\newcommand{\rjup}{R_\mathrm{J}}$
$\newcommand{\mjup}{M_\mathrm{J}}$
$\newcommand{\Porb}{P_{\rm orb}}$
$\newcommand{\Prot}{P_{\rm rot}}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\lstar}{L_\star}$
$\newcommand{\rstar}{R_\star}$
$\newcommand{\rhostar}{\rho_{\mathrm{*}}}$
$\newcommand{\rhoplanet}{\rho_\mathrm{p}}$
$\newcommand{\rplanet}{R_{\mathrm{p}}}$
$\newcommand{\mplanet}{M_{\mathrm{p}}}$
$\newcommand{\halpha}{H\alpha}$
$\newcommand{\hbeta}{H\beta}$
$\newcommand{\sii}{[S {\sc II}] }$
$\newcommand{\oii}{[O {\sc I}] }$
$\newcommand{\oiii}{[O {\sc III}] }$
$\newcommand{\oi}{[O {\sc i}] }$
$\newcommand{\cai}{Ca {\sc I} }$
$\newcommand{\caii}{[Ca {\sc II}] }$
$\newcommand{\feii}{[Fe {\sc II}] }$
$\newcommand{\nii}{[N {\sc II}] }$
$\newcommand{\mgi}{Mg {\sc I} }$
$\newcommand{\nai}{[Na {\sc I}]}$
$\newcommand{\av}{A_{V}}$
$\newcommand{\vmag}{m_{V}}$
$\newcommand{\tic}{TIC~229650439}$
$\newcommand{\ticno}{229650439}$
$\newcommand{\ticb}{TIC~229650439.01}$
$\newcommand{\ticc}{TIC~229650439.02}$
$\newcommand{\targeta}{\mbox{TOI-1438}}$
$\newcommand{\targetone}{TOI-1438.01}$
$\newcommand{\targettwo}{TOI-1438.02}$
$\newcommand{\targetb}{TOI-1438~b}$
$\newcommand{\targetc}{TOI-1438~c}$
$\newcommand{\bjdtdb}{\ensuremath{\rm{BJD_{TDB}}}}$
$\newcommand{\logRHK}[1][]{-4.925 \pm 0.013}$
$\newcommand{\smassariadne}[1][]{0.876 \pm 0.038}$
$\newcommand{\smassariadnegrav}[1][]{0.764\pm 0.127}$
$\newcommand{\smassparam}[1][]{0.867 \pm 0.027}$
$\newcommand{\sradiusariadne}[1][]{0.820 \pm 0.017}$
$\newcommand{\sradiusisochrones}[1][]{0.x\pm 0.0x}$
$\newcommand{\sradiusparam}[1][]{0.822\pm 0.018}$
$\newcommand{\sradiusspechmatch}[1][]{0.81\pm 0.08}$
$\newcommand{\sradiusgaia}[1][]{0.853^{+0.026}_{-0.016} }$
$\newcommand{\srhoariadne}[1][]{2.2 \pm 0.2}$
$\newcommand{\srhospechmatch}[1][]{x \pm x}$
$\newcommand{\srhoparam}[1][]{2.2 \pm 0.2}$
$\newcommand{\srhotransitmodel}[1][]{x \pm x}$
$\newcommand{\srhotic}[1][]{x \pm x}$
$\newcommand{\steffariadne}[1][]{5230\pm 60}$
$\newcommand{\sloggariadne}[1][]{4.50\pm0.07}$
$\newcommand{\sfehariadne}[1][]{0.03\pm0.06}$
$\newcommand{\Lumariadne}[1][]{0.45 \pm 0.02}$
$\newcommand{\Avariadne}[1][]{0.04 \pm 0.05}$
$\newcommand{\steffsme}[1][]{5189 \pm 65}$
$\newcommand{\sloggsme}{4.50\pm 0.05}$
$\newcommand{\scahsme}[1][]{0.07\pm 0.06}$
$\newcommand{\sfehsme}[1][]{0.04\pm 0.05}$
$\newcommand{\snasme}[1][]{0.13\pm 0.06}$
$\newcommand{\smghsme}[1][]{0.14 \pm 0.06}$
$\newcommand{\ssihsme}[1][]{0.07 \pm 0.06}$
$\newcommand{\svsinisme}[1][]{1.8 \pm 0.9}$
$\newcommand{\svmic}[1][]{0.8}$
$\newcommand{\svmac}[1][]{2.5}$
$\newcommand{\velsme}[1][\mathrm{km s^{-1}}]{-29.21}$
$\newcommand{\steffspechmatch}[1][]{5225 \pm 110}$
$\newcommand{\sloggspechmatch}[1][]{4.53 \pm 0.12}$
$\newcommand{\sfehspechmatch}[1][]{0.06\pm 0.09}$
$\newcommand{\sloggparam}[1][]{4.52 \pm 0.03}$
$\newcommand{\steffgaia}[1][]{5202^{+8}_{-4} }$
$\newcommand{\slogggaia}[1][]{4.53 \pm 0.01}$
$\newcommand{\sfehgaia}[1][]{-0.14 \pm 0.01}$
$\newcommand{\lstargaia}[1][]{0.449\pm0.002}$
$\newcommand{\distancegaia}{ 110.7\pm 0.1}$
$\newcommand{\parallaxgaia}{9.0312\pm0.0111}$
$\newcommand{\velgaia}[1][\mathrm{km s^{-1}}]{-29.40 \pm 0.34}$
$\newcommand{\pmra}{-22.8246 \pm 0.0125}$
$\newcommand{\pmdec}{53.0721 \pm  0.0155}$
$\newcommand{\stefftfop}[1][]{5259\pm 125}$
$\newcommand{\sfehtfop}[1][]{0.10 \pm 0.06 }$
$\newcommand{\sloggtfop}[1][]{4.57 \pm 0.09 }$
$\newcommand{\sradiustfop}[1][]{0.816 \pm 0.046 }$
$\newcommand{\smasstfop}[1][]{0.90 \pm 0.11}$
$\newcommand{\spectraltype}{K0V}$
$\newcommand{\smassspectraltype}{0.88}$
$\newcommand{\sradiusspectraltype}{0.81}$
$\newcommand{\srhospectraltype}{2.3}$
$\newcommand{\protvsini}{22.8 \pm 11.4}$
$\newcommand{\prottess}{23.0 \pm 1.4}$
$\newcommand{\agespechmatch}[1][]{4.2\pm 1.5}$
$\newcommand{\ageparam}{5.9\pm 4.2}$
$\newcommand{\ageariadne}[1][]{2.6^{+4.2}_{-2.3}}$
$\newcommand{\ageallmodels}[1][]{2 - 6}$
$\newcommand{\Tzerob}[1][]$
$\newcommand{\Tzeroc}[1][]$
$\newcommand{\Tzerod}[1][]$
$\newcommand{\Pb}[1][]$
$\newcommand{\Pc}[1][]$
$\newcommand{\lnPd}[1][]$
$\newcommand{\Pd}[1][]$
$\newcommand{\Pdyear}[1][]$
$\newcommand{\eb}[1][ ]$
$\newcommand{\ebfree}[1][ ]$
$\newcommand{\wb}[1][]$
$\newcommand{\cosib}[1][]$
$\newcommand{\bb}[1][ ]$
$\newcommand{\arb}[1][ ]$
$\newcommand{\rrb}[1][ ]$
$\newcommand{\kb}[1][]$
$\newcommand{\ecfree}[1][ ]$
$\newcommand{\wc}[1][]$
$\newcommand{\ed}[1][ ]$
$\newcommand{\wwd}[1][]$
$\newcommand{\cosic}[1][]$
$\newcommand{\bc}[1][ ]$
$\newcommand{\arc}[1][ ]$
$\newcommand{\rrc}[1][ ]$
$\newcommand{\kc}[1][]$
$\newcommand{\kd}[1][]$
$\newcommand{\mpb}[1][]$
$\newcommand{\mpc}[1][]$
$\newcommand{\mpdearth}[1][]$
$\newcommand{\mpdjup}[1][]$
$\newcommand{\rpb}[1][]$
$\newcommand{\rpc}[1][]$
$\newcommand{\Tperib}[1][]$
$\newcommand{\ecosd}[1][]$
$\newcommand{\esind}[1][]$
$\newcommand{\ecosc}[1][]$
$\newcommand{\esinc}[1][]$
$\newcommand{\eblim}[1][]$
$\newcommand{\eclim}[1][]$
$\newcommand{\prvb}[1][]$
$\newcommand{\ib}[1][]$
$\newcommand{\ic}[1][]$
$\newcommand{\ab}[1][]$
$\newcommand{\ac}[1][]$
$\newcommand{\ad}[1][]$
$\newcommand{\depthbSC}[1][]$
$\newcommand{\RMbSC}[1][]$
$\newcommand{\insolationb}[1][]$
$\newcommand{\insolationc}[1][]$
$\newcommand{\tsmb}[1][ ]$
$\newcommand{\tsmc}[1][ ]$
$\newcommand{\denstrb}[1][]$
$\newcommand{\densspb}[1][]$
$\newcommand{\Teqb}[1][]$
$\newcommand{\Teqc}[1][]$
$\newcommand{\ttotb}[1][]$
$\newcommand{\tfulb}[1][]$
$\newcommand{\ttotc}[1][]$
$\newcommand{\tfulc}[1][]$
$\newcommand{\tegb}[1][]$
$\newcommand{\deltamagb}[1][]$
$\newcommand{\denpb}[1][]$
$\newcommand{\denpc}[1][]$
$\newcommand{\grapb}[1][]$
$\newcommand{\grapc}[1][]$
$\newcommand{\grapparsb}[1][]$
$\newcommand{\jspb}[1][]$
$\newcommand{\jspc}[1][]$
$\newcommand{\qoneSC}[1][]$
$\newcommand{\qtwoSC}[1][]$
$\newcommand{\qsum}[1][]$
$\newcommand{\HARPSN}[1][]$
$\newcommand{\jHARPSN}[1][]$
$\newcommand{\HIRES}[1][]$
$\newcommand{\jHIRES}[1][]$</div>



<div id="title">

# TOI-1438:  A  rare system with two short-period sub-Neptunes and a tentative long-period Jupiter-like planet  orbiting a  $\spectraltype$ star

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.21533-b31b1b.svg)](https://arxiv.org/abs/2508.21533)<mark>Appeared on: 2025-09-01</mark> -  _31 pages, 21 figures. Accepted by Astronomy & Astrophysics (A&A) 31 July 2025_

</div>
<div id="authors">

C. M. Persson, et al. -- incl., <mark>I. J. M. Crossfield</mark>

</div>
<div id="abstract">

**Abstract:** We present the detection and characterisation of the $\targeta$ multi-planet system   discovered   by the  Transiting Exoplanet Survey Satellite (TESS).To confirm the planetary nature of the  candidates and determine their masses,  we collected a series  of follow-up observations including    high-spectral resolution  observations with HARPS-N and HIRES over a period of five years.  Our combined modelling  shows that the $\spectraltype$ star hosts   two transiting sub-Neptunes with $R_\mathrm{b} = $ $\rpb$  $\rearth$ , $R_\mathrm{c} = $ $\rpc$  $\rearth$ , $M_\mathrm{b} = $ $\mpb$  $\mearth$ , and $M_\mathrm{c} = $ $\mpc$  $\mearth$ . The orbital periods of planets b and c are 5.1 and 9.4 days, respectively,   corresponding to instellations of $\insolationb$  $\Fearth$ and $\insolationc$  $\Fearth$ .  The bulk densities are $\denpb$  $\gc$ and $\denpc$  $\gc$ , respectively, suggesting a volatile-rich   interior composition. By combining the planet and stellar parameters, we were able to compute a set of  planet interior structure models. Planet b presents a high-metallicity envelope that can accommodate up to 2.5 \% in H/He in mass, while planet c cannot have more than 0.2 \% as H/He in mass. For any composition of the core considered (Fe-rock or ice-rock), both planets would require a volatile-rich envelope.  In addition to the two planets, the radial velocity (RV) data  clearly reveal a third   signal,   likely coming from a non-transiting planet, with an   orbital period of $\Pdyear$ years  and an RV semi-amplitude of $\kd$  $\ms$ . Our best-fit model finds a minimum mass of $\mpdjup$  $\mjup$ and an eccentricity of $\ed$ .However, several RV activity indicators also show  strong signals at similar periods, suggesting  this signal might (partly) originate from   stellar   activity.More data over a longer period of time are needed to conclusively determine the nature of this   signal.If it is confirmed as a triple-planet system, $\targeta$ would be one of the few detected systems to date characterised by an architecture with two small, short-period planets and one massive, long-period planet, where the inner and outer systems are separated by an orbital period ratio of the order of a few hundred.

</div>

<div id="div_fig1">

<img src="tmp_2508.21533/./Fig08a.png" alt="Fig6.1" width="25%"/><img src="tmp_2508.21533/./Fig08b.png" alt="Fig6.2" width="25%"/><img src="tmp_2508.21533/./Fig08c.png" alt="Fig6.3" width="25%"/><img src="tmp_2508.21533/./Fig08d.png" alt="Fig6.4" width="25%"/>

**Figure 6. -** Dynamical maps of the eccentricities of the two inner planets ($e_b$, $e_c$) and fixed values of the argument of pericentre of $\targetb$($\Delta \varpi =\omega_c-\omega_b$). The black filled circle with a white rim shows the location of the solution obtained by modelling with the eccentricities set as free parameters. Small values of the fast indicator $\log \mathrm{\widehat{REM}}$ characterises regular (long-term stable) solutions, which are marked with black and dark blue colour. Chaotic solutions are marked with brighter colours, up to yellow. The black line represents the so-called collision curve of orbits, defined by the condition: $a_b(1 + e_b) = a_c(1 - e_c)$.  The resolution of each plot is 301 $\times$ 301 points. Triangles  marked $1-3$ correspond to the solutions shown in Fig. \ref{figure-TOI-2427-ebec}.
    Also, the labelled grey contours refer to the TTV amplitudes shown in Fig. \ref{figure-TOI-2427-TTV}(details in Section \ref{sec-dynamical_analysis}).
     (*figure-TOI-2427-dynamical_map*)

</div>
<div id="div_fig2">

<img src="tmp_2508.21533/./Fig14.png" alt="Fig10" width="100%"/>

**Figure 10. -** RV jitter ($\sigma_1$) as derived in \citet{Bonomo2017} as a function of $\log R^\prime_{\rm HK}$ activity index calculated in \citet{Claudi2024} for exoplanet host stars with $M_\star<1.1$ $\Msun$. Light-grey error bars denote systems with a baseline shorter than 1 yr, grey error bars are systems with a baseline  $1-2$ yr, and black error bars are systems with   baselines $>2$ yr. The error bars with arrows are upper limits on the RV jitter. The markers are colour coded with the stellar $T_{\rm eff}$. $\targeta$ is marked with a square. The marked systems HAT-P-15, HAT-P-19, TrES-4, and WASP-10 are discussed in Sect. \ref{Subsection: system architecture}.
        Adapted from Fig. 3 in  \citet{Hekker2006}. (*fig:jitter*)

</div>
<div id="div_fig3">

<img src="tmp_2508.21533/./Fig07.png" alt="Fig13" width="100%"/>

**Figure 13. -** RVs of $\targeta$. Top: The HARPS-N (blue) and HIRES (orange) RV time series with the best-fitting three Keplerian model in grey with residuals shown in the panel below. Lower: The phasefolded RV curves for planet b (left) and c (middle) as well as signal d (right). The best-fitting models are shown as the grey lines with the shaded area denoting the $1$ and $2 \sigma$ intervals in the $K$-amplitude. Residuals are given below. (*fig:rvs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.21533"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\oii}{O {\scriptsize II}}$
$\newcommand{\oiii}{O {\scriptsize III}}$
$\newcommand{\oiv}{O {\scriptsize IV}}$
$\newcommand{\caii}{Ca {\scriptsize II}}$
$\newcommand{\cii}{C {\scriptsize II}}$
$\newcommand{\ciii}{C {\scriptsize III}}$
$\newcommand{\sii}{S {\scriptsize II}}$
$\newcommand{\nii}{N {\scriptsize II}}$
$\newcommand{\niii}{N {\scriptsize III}}$
$\newcommand{\niv}{N {\scriptsize IV}}$
$\newcommand{\neiii}{Ne {\scriptsize III}}$
$\newcommand{\nv}{N {\scriptsize V}}$
$\newcommand{\civ}{C {\scriptsize IV}}$
$\newcommand{\hi}{H {\scriptsize I}}$
$\newcommand{\hii}{H {\scriptsize II}}$
$\newcommand{\hei}{He {\scriptsize I}}$
$\newcommand{\heii}{He {\scriptsize II}}$
$\newcommand{\feiv}{Fe {\scriptsize IV}}$
$\newcommand{\nev}{Ne {\scriptsize V}}$
$\newcommand{\hdelta}{H\delta}$
$\newcommand{\hbeta}{H\beta}$
$\newcommand{\halpha}{H\alpha}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\hst}{\textit{HST}}$
$\newcommand{\jwst}{\textit{JWST}}$
$\newcommand{\spitzer}{\textit{Spitzer}}$
$\newcommand{\eazy}{\texttt{EAzY}}$
$\newcommand{\pypeit}{\texttt{PypeIt}}$
$\newcommand{\bagpipes}{\texttt{Bagpipes}}$
$\newcommand{\thefigure}{A\arabic{figure}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# JWST Spectroscopic Insights Into the Diversity of Galaxies in the First 500 Myr: Short-Lived Snapshots Along a Common Evolutionary Pathway

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.21708-b31b1b.svg)](https://arxiv.org/abs/2508.21708)<mark>Appeared on: 2025-09-01</mark> -  _20 pages, 13 figures. Submitted for publication, comments welcome_

</div>
<div id="authors">

G. Roberts-Borsani, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We investigate the nature and spectroscopic diversity of early galaxies from a sample of 40 sources at $z\geqslant10$ with JWST/NIRSpec prism observations, the largest of its kind thus far. We compare the properties of strong UV line emitters, as traced by intense $\civ$ emission, with those of more "typical" sources with weak or undetected $\civ$ . The more typical (or " $\civ$ -weak") sources reveal significant scatter in their $\ciii$ ] line strengths, UV continuum slopes, and physical sizes, spanning $\ciii$ ] equivalent widths of $\sim$ 1-51 Å, UV slopes of $\beta\sim-1.6$ to $-2.6$ , and half-light radii of $\sim$ 50-1000 pc. In contrast, $\civ$ -strong sources generally occupy the tail of these distributions, with $\ciii$ ] EWs of 16-51 Å, UV slopes $\beta\lesssim-2.5$ , compact morphologies ( $r_{\rm 50} \lesssim 100$ pc), and elevated star formation surface densities ( $\Sigma_{\rm SFR} \gtrsim 100 M_\odot \mathrm{yr}^{-1} \mathrm{kpc}^{-2}$ ). Collectively these properties are consistent with concentrated starbursts that temporarily outshine the extended structure of the galaxy. Comparing average properties from composite spectra, we find the diversity of the sample is primarily driven by bursts and lulls of star formation on very short timescales ( $\leq$ 3 Myr), where strong $\civ$ emitters are observed at the apex of these phases and sources devoid of emission lines represent periods of relative inactivity. An apparent association between strong $\civ$ and enhanced nitrogen abundance suggests both features may be modulated by the same duty cycle and reflect a generic mode of star formation. We show that AGN are unlikely to be a significant contributor to this duty cycle based on a comparison of UV line diagnostics to photoionisation models, although some non-thermal activity cannot be fully ruled out. Our results support a unified evolutionary picture whereby transient bursts and lulls can explain the spectral diversity and early growth of bright galaxies in the first 500 Myr.

</div>

<div id="div_fig1">

<img src="tmp_2508.21708/./figures/stack_spectrum.png" alt="Fig12" width="100%"/>

**Figure 12. -** **Composite spectra of the $\civ$-strong (dark blue) and $\civ$-weak (light blue) samples at $z\simeq10.0-14.4$, and the corresponding 2D spectrum of the former.** Emission lines of interest (detected or not) and the Ly$\alpha$ break are indicated by red vertical lines in the 1D spectrum, as well as S/N ratio contours of 3, 5 and 10 in the 2D spectrum. The $\civ$-strong composite reveals strongly enhanced $\niv$], $\civ$, $\hei$i$+$\oiii], and $\cii$i] line emission compared to its $\civ$-weak counterpart comprising the rest of the sample. The comparison highlights the strong variance across the galaxy population and diversity of astrophysical processes underpinning the evolution of galaxies within the first 500 Myr. (*fig:stack*)

</div>
<div id="div_fig2">

<img src="tmp_2508.21708/./figures/uvslope_origins.png" alt="Fig6" width="100%"/>

**Figure 6. -** **Contributions to the UV continuum slopes of $z\geqslant10$ galaxies.** The mass-weighted ages from \texttt{Bagpipes} are plotted with the spectroscopic $\beta$ measurements for individual objects (small squares) and composite measurements (stars), following previous color schemes. Composite symbols are scaled in size according to their recent star formation activity, where a larger size equates to more recent star formation. The intrinsic scatter in $\beta$ measured in Section \ref{subsec:beta} for $\civ$-weak objects is shown as the light-blue box, while the range of $\beta$ determined from a number of FSPS models are shown as green fill (varying metallicity), red fill (timing relative to a burst of star formation), purple fill (stellar vs nebular continuum contributions), and grey fill (varying stellar ages). Only variations in the recent star formation history and in stellar age are able to explain the intrinsic scatter of observed UV slopes. (*fig:beta_origins*)

</div>
<div id="div_fig3">

<img src="tmp_2508.21708/./figures/bursty.png" alt="Fig7" width="100%"/>

**Figure 7. -** **The burstiness of star formation in $z\geqslant10$ galaxies.** Individual measurements are plotted as small squares following previous color schemes, while composite measurements from stacked NIRSpec spectra and NIRCam F200W data of _unlensed_ sources are shown as stars (grey for the $\civ$-weak population, gold for the $\civ$-strong population). Each composite symbol is scaled in size according to their recent star formation activity, where larger size corresponds to more recent star formation. The log-normal scatter around the median of the morphological distributions in Section \ref{subsec:morphologies} are plotted as light blue boxes. The large range of SFR surface densities and correlation with half-light radius suggests varying gas densities and star formation efficiencies, while the positions of $\civ$-strong objects at the most compact end of the distribution with high star formation rate surface densities highlights the requirement of extreme star formation episodes to power $\civ$ emission. (*fig:bursty*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.21708"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xHI}{\langle x_\mathrm{HI} \rangle}$
$\newcommand{\NHI}{N_\mathrm{HI}^\mathrm{DW}}$
$\newcommand{\NHIabs}{N_\mathrm{HI}^\mathrm{abs}}$
$\newcommand{\NHIIGM}{N_\mathrm{HI}^\mathrm{IGM}}$
$\newcommand{\Rb}{r_\mathrm{patch}}$
$\newcommand{\tQ}{t_\mathrm{Q}}$
$\newcommand{\logNHI}{\log_{10} \NHI/\mathrm{cm}^{-2}}$
$\newcommand{\logNHIabs}{\log\NHIabs}$
$\newcommand{\logNHIIGM}{\log\NHIIGM}$
$\newcommand{\logtQ}{\log_{10} \tQ/\mathrm{yr}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# First constraints on the local ionization topology in front of two quasars at $z \sim 7.5$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.21818-b31b1b.svg)](https://arxiv.org/abs/2508.21818)<mark>Appeared on: 2025-09-01</mark> -  _submitted to MNRAS_

</div>
<div id="authors">

T. Kist, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** Thus far, Lyman- $\alpha$ damping wings towards quasars have been used to probe the _global_ ionization state of the foreground intergalactic medium (IGM). A new parameterization has demonstrated that the damping wing signature also carries _local_ information about the distribution of neutral hydrogen (HI) in front of the quasar before it started shining. Leveraging a recently introduced Bayesian \texttt{JAX} -based Hamiltonian Monte Carlo (HMC) inference framework, we derive constraints on the Lorentzian-weighted HI column density $\NHI$ , the quasar's distance $\Rb$ to the first neutral patch and its lifetime $\tQ$ based on JWST/NIRSpec spectra of the two $z \sim 7.5$ quasars J1007+2115 and J1342+0928. After folding in model-dependent topology information, we find that J1007+2115 (and J1342+0928) is most likely to reside in a $\xHI = 0.32_{-0.20}^{+0.22}$ ( $0.58_{-0.23}^{+0.23}$ ) neutral IGM while shining for a remarkably short lifetime of $\logtQ = 4.14_{-0.18}^{+0.74}$ (an intermediate lifetime of $5.64_{-0.43}^{+0.25}$ ) along a sightline with $\logNHI = 19.70_{-0.86}^{+0.35}$ ( $20.24_{-0.22}^{+0.25}$ ) and $\Rb = 28.9_{-14.4}^{+54.0}  \mathrm{cMpc}$ ( $10.9_{-5.9}^{+5.6}  \mathrm{cMpc}$ ). In light of the potential presence of local absorbers in the foreground of J1342+0928 as has been recently suggested, we also demonstrate how the Lorentzian-weighted column density $\NHI$ provides a natural means for quantifying their contribution to the observed damping wing signal.

</div>

<div id="div_fig1">

<img src="tmp_2508.21818/./figures/mcmc_proc_JWST_J1007+2115_iqso_000.hdf5_P_xHI_and_tQ_and_NHI_and_Rb_.png" alt="Fig1" width="100%"/>

**Figure 1. -** Posterior distributions inferred from the JWST/NIRSpec spectrum of the quasar J1007+2115 depicted in Figure \ref{fig:spec_J1007} in the context of the local IGM damping wing parameterization (black) and the global one (yellow). Both distributions are marginalized over $7$ nuisance parameters describing the shape of the quasar continuum. Additionally depicted in green is the topology-informed version of the local constraints, entailing the non-trivial prior $P_\mathrm{top}($\logNHI$, $\Rb$)$(explicitly depicted in purple in the extra panel), and also providing a constraint on the global IGM neutral fraction $\xHI$ in good agreement with the directly inferred one. (*fig:corner_J1007*)

</div>
<div id="div_fig2">

<img src="tmp_2508.21818/./figures/mcmc_proc_JWST_J1007+2115_spec_iqso_000_.png" alt="Fig4" width="100%"/>

**Figure 4. -** Inferred model for the JWST/NIRSpec spectrum of the quasar J1007+2115, fitted in the context of the local IGM damping wing parameterization (upper row) and the global one (lower row). The observed (and rebinned) spectrum of the quasar is depicted in black, with the noise vector shown in yellow. The inferred model spectrum is depicted in red and the unabsorbed inferred continuum in blue, where solid lines represent the median inferred models, and shaded regions the $16 \%$ and the $84 \%$ percentile variations reflecting parameter uncertainty, continuum reconstruction errors, as well as spectral noise. (*fig:spec_J1007*)

</div>
<div id="div_fig3">

<img src="tmp_2508.21818/./figures/mcmc_proc_JWST_J1342+0928_iqso_000.hdf5_P_xHI_and_tQ_and_NHI_and_Rb_.png" alt="Fig2" width="100%"/>

**Figure 2. -** Like Figure \ref{fig:corner_J1007} but for the posterior distributions inferred from the JWST/NIRSpec spectrum of the quasar J1342+0928 depicted in Figure \ref{fig:spec_J1342}. The $($\NHI$, $\Rb$)$ panel also shows the putative local absorber constraints obtained by davies2025 assuming two different metallicities which provide lower limits on $\NHI$. (*fig:corner_J1342*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.21818"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

129  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
